In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo",]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

### [1. Forward Imaging Model](fwdimaging.ipynb)

## 2. The Recovery Process
This process recover the Hi-Res complex object using the Lo-Res image set. Each image of it is generated by the lighting that emerges from a different LED of a grid: the forward imaging process ([fwdimaging.ipynb](fwdimaging.ipynb)).

In [ ]:
import fwdimaging as fwd
import numpy as np
import scipy.misc
import matplotlib.pyplot as plt
import os

In [ ]:
def get_sequence(leds_prc, **kwargs):
    """Return a list with the indexes of the images for the
    reconstruction sequence.
    
    The way is spiral-shaped. For square arrays with odd number of
    rows, the spiral's head is the center; for even number of rows,
    the (center + 1, center - 1).
    """
    clockwise = kwargs.pop('clockwise', True)
    n2max = kwargs.pop('n2max', leds_prc ** 2 - 1)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(get_idx_seq_img.__name__, list(kwargs.keys())[-1]))
    l_idx_seq_img = list()
    if leds_prc % 2 != 0:
        img = int(((leds_prc ** 2) - 1) / 2)  # Start in the center.
    else:
        img = int(((leds_prc ** 2) - leds_prc) / 2)
    l_idx_seq_img.append(img)
    l_addends = [(leds_prc, -1), (-leds_prc, 1)]
    if not clockwise:
        l_addends = [(leds_prc, 1), (-leds_prc, -1)]
    # Index to choose between (leds_prc, -1) ('tuple') and
    # (-leds_prc, 1).
    i = 0
    c = 1
    while True:
        i %= 2
        v_addend, h_addend = l_addends[i]
        for j in range(c):
            img += v_addend
            if img > n2max or img < 0 or img in l_idx_seq_img:
                return l_idx_seq_img
            else:
                l_idx_seq_img.append(img)
        for j in range(c):
            img += h_addend
            if img > n2max or img < 0 or img in l_idx_seq_img:
                return l_idx_seq_img
            else:
                l_idx_seq_img.append(img)
        i += 1
        c += 1

In [ ]:
def get_photo(**kwargs):
    """Return the Hi-Res object recovered by applying of Fourier
    Ptychography algorithm.
    
    In recovery by importing of an externel set, i.e., sim = False,
    it is possible to take a lower quantity of pictures of the
    complete set.
    """
    inpath = kwargs.pop('inpath', 'img-lores/')
    inprefix = kwargs.pop('inprefix', 'lores_')
    inextension = kwargs.pop('inextension', '.tif')
    firstsuffix = kwargs.pop('firstsuffix', '001')
    outpath = kwargs.pop('outpath', 'out-img-hires/')
    # n ('int') is the number of LEDs per row and column. It could be
    # simple remember the question "how many rows of LEDs does my grid
    # have?". The array will be n-by-n order.
    # A LED grid looks like the Adafruit NeoPixel NeoMatrix 8x8 
    # (https://www.adafruit.com/product/1487). n ('int') is useful
    # only in simulation mode.
    n = kwargs.pop('n', 15)
    # n2use ('int') IS ONLY USEFUL when you have an external Lo-Res
    # image set and you want to select just a few pictures captured by
    # a m-by-m subset of LED in which m is less than n in the n-by-n
    # order of the complete grid.
    n2use = kwargs.pop('n2use', 15)
    d = kwargs.pop('d', 4)  # Distance in mm between LEDs.
    # Distance in mm between the LED grid and the sample.
    h = kwargs.pop('h', 90)
    # Indexes in Matlab begin in 1. Does the first image have 1 at the
    # end of its filename?
    namefrom1 = kwargs.pop('namefrom1', True)
    subsel = kwargs.pop('subsel',
                        fwd.get_centered_img_subset(n,
                                                    n2use,
                                                    matlab=namefrom1))
    ###
    # Simulate the set of Lo-Res images.
    sim = kwargs.pop('sim', False)
    # Only create the Lo-Res image set and do not do the reconstruction
    # of the Hi-Res picture.
    justset = kwargs.pop('justset', False)
    amplitude = kwargs.pop('amplitude',
                           'img/Alberto-Romero_Tornillos_flickr_1024x683.tif')
    phase = kwargs.pop('phase',
                       'img/pabloaez_Valparaiso-054_flickr_4000x3000.png')
    ###
    # Parameters of the coherent imaging system.
    wavelen = kwargs.pop('wavelen', 0.63e-6)
    # Sampling pixel size of the CCD.
    ccdpx = kwargs.pop('ccdpx', 2.75e-6)
    # Pixel size of the reconstruction.
    hirespx = kwargs.pop('hirespx', ccdpx / 4)
    na = kwargs.pop('na', 0.08)  # Numerical aperture of the employed
                                 # objective lens.
    # DANGER! IT COULD PIXELATE the output image.
    zoom = kwargs.pop('zoom', 4)
    ###
    loops = kwargs.pop('loops', 5)
    # Show the output Hi-Res image in the notebook.
    name = kwargs.pop('name', 'out-img-hires')  # Name of the file.
    extension = kwargs.pop('extension', '.tiff')
    # Does it the output image in the notebook of the calling?
    show = kwargs.pop('show', False)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(get_photo.__name__, list(kwargs.keys())[-1]))
    if sim and n2use != n:
        n2use = n
    wvs = fwd.generate_wave_vectors(n2use, d, h)
    if sim:
        in_obj = fwd.generate_obj(amplitude, phase)
        hpx, wpx = in_obj.shape
        p, q, dkx, dky, kx, ky, cft = fwd.get_cft(wvs, wavelen, ccdpx,
                                                  zoom, na, wpx, hpx)
        img_seq_lores = fwd.generate_lores_set(in_obj, cft, n, kx, ky,
                                               wpx, hpx, dkx, dky, p,
                                               q)
        if justset:
            return img_seq_lores
    else:  # Import your own Lo-Res image set.
        img_seq_lores = fwd.get_set_from_folder(subsel, path=inpath,
                                                prefix=inprefix,
                                                extension=inextension,
                                                lensuffix=len(firstsuffix))
        # Search for the height and width in pixels of a Lo-Res
        # picture.
        hpx = 0
        wpx = 0
        for i in range(n2use ** 2):
            if len(img_seq_lores[i]) != 0:
                hpx, wpx = img_seq_lores[i].shape
                break
        # np.zeros((0, 0)) arrays in img_seq_lores ('numpy.ndarray')
        # are replaced by other array from the same kind and
        # wpx-by-hpx shape.
        for i in range(n2use ** 2):
            if len(img_seq_lores[i]) == 0:
                img_seq_lores[i] = np.zeros((hpx, wpx))
        hpx *= zoom  # Height of the output Hi-Res image.
        wpx *= zoom  # ... the width.
        p, q, dkx, dky, kx, ky, cft = fwd.get_cft(wvs, wavelen, ccdpx,
                                                  zoom, na, wpx, hpx)
    # For a shifted subset, the -1 values will also be considered 
    seq = get_sequence(n2use)
    obj_recover = np.ones((hpx, wpx), dtype=np.int)
    obj_recover_ft = np.fft.fftshift(np.fft.fft2(obj_recover))
    for i in range(loops):
        for j in range(n2use ** 2):
            k = seq[j]
            kxc = fwd.round_half_up((wpx + 1) / 2.0 + kx[k] / dkx)
            kyc = fwd.round_half_up((hpx + 1) / 2.0 + ky[k] / dky)
            kyl = fwd.round_half_up(kyc - (p - 1) / 2.0)
            kyh = fwd.round_half_up(kyc + (p - 1) / 2.0)
            kxl = fwd.round_half_up(kxc - (q - 1) / 2.0)
            kxh = fwd.round_half_up(kxc + (q - 1) / 2.0)
            lores_ft = ((p / hpx) ** 2) * obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh]
            lores_ft *= cft
            img_lores = np.fft.ifft2(np.fft.ifftshift(lores_ft))
            img_lores = ((hpx / p) ** 2) * img_seq_lores[k] * np.exp(1j * np.angle(img_lores))
            lores_ft = np.fft.fftshift(np.fft.fft2(img_lores)) * cft
            obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] = (1 - cft) * obj_recover_ft[kyl - 1:kyh, kxl - 1:kxh] + lores_ft
    obj_recover = np.fft.ifft2(np.fft.ifftshift(obj_recover_ft))
    obj_recover = np.absolute(obj_recover)
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    scipy.misc.imsave(outpath + name + extension, obj_recover)
    if show:
        plt_img = plt.imshow(obj_recover, cmap='Greys_r')
    return obj_recover